# 1. plylst를 이용한 태그 유사도 학습

In [1]:
import pandas as pd
import numpy as np

In [2]:
Data = pd.read_json('train.json')
Data.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [3]:
len(Data)

115071

In [102]:
val_Data = pd.read_json('val.json')
val_Data.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [108]:
# val_Data 중 태그가 있는 항목 출력
val_TagIndex_list = []
for i in range(len(val_Data)):
    if val_Data.iloc[i]['tags'] != []:
        val_TagIndex_list.append(i)
val_TagIndex_list

[5,
 8,
 11,
 12,
 17,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 30,
 31,
 34,
 38,
 40,
 42,
 43,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 55,
 56,
 58,
 59,
 62,
 64,
 65,
 67,
 68,
 69,
 70,
 74,
 75,
 77,
 78,
 80,
 81,
 82,
 84,
 87,
 91,
 92,
 94,
 95,
 96,
 100,
 104,
 105,
 108,
 109,
 112,
 113,
 114,
 116,
 122,
 124,
 125,
 127,
 128,
 129,
 130,
 134,
 136,
 139,
 140,
 141,
 144,
 145,
 148,
 151,
 158,
 160,
 161,
 162,
 164,
 165,
 168,
 169,
 172,
 177,
 181,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 193,
 194,
 196,
 200,
 203,
 207,
 208,
 209,
 211,
 212,
 215,
 219,
 222,
 223,
 225,
 226,
 230,
 231,
 232,
 234,
 235,
 238,
 239,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 255,
 258,
 259,
 261,
 262,
 266,
 267,
 269,
 273,
 274,
 276,
 277,
 278,
 281,
 283,
 284,
 285,
 288,
 295,
 296,
 297,
 299,
 302,
 305,
 310,
 312,
 313,
 314,
 315,
 318,
 320,
 322,
 324,
 326,
 329,
 332,
 335,
 337,
 338,
 339,
 340,
 341,
 342,
 345,
 346,
 349,
 351,
 353,
 35

,tags,id,plylst_title,songs,like_cnt,updt_date
5,"[비, 분위기, 컨트리, 서정적인]",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
11,"[아침, 눈, 크리스마스]",136650,,"[174132, 257367, 539802, 568691, 296891, 38471...",6,2017-11-24 11:26:05.000
12,[EDM],53131,,"[392257, 668879, 460746, 680240, 490966, 25010...",268,2016-09-19 15:40:31.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
...,...,...,...,...,...,...
23007,"[휴식, 포크, 커피, 2000년대]",44013,,"[434841, 362936, 361453, 628693, 113135, 35071...",180,2019-12-17 17:55:11.000
23008,"[sweetmind, 발라드, 사랑, 기분전환, 역주행]",100684,[별책부록] Sweetmind의 음악마중,[],79,2020-02-07 14:42:25.000
23009,[알앤비],17766,,"[114387, 701557, 547449, 473514, 360916, 49845...",1,2015-07-21 17:06:45.000
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000


,id,plylst_title,tags,songs
5,138538,,"[비, 분위기, 컨트리, 서정적인]","[173033, 138237, 63140, 15733, 461780, 308519,..."
8,80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[스트레스],[]
11,136650,,"[아침, 눈, 크리스마스]","[174132, 257367, 539802, 568691, 296891, 38471..."
12,53131,,[EDM],"[392257, 668879, 460746, 680240, 490966, 25010..."
17,2380,다시 생각나는 그 사람,[생각나],[]
...,...,...,...,...
23007,44013,,"[휴식, 포크, 커피, 2000년대]","[434841, 362936, 361453, 628693, 113135, 35071..."
23008,100684,[별책부록] Sweetmind의 음악마중,"[sweetmind, 발라드, 사랑, 기분전환, 역주행]",[]
23009,17766,,[알앤비],"[114387, 701557, 547449, 473514, 360916, 49845..."
23010,101722,,[잔잔한],"[75842, 26083, 244183, 684715, 500593, 508608,..."


In [114]:
plylst_tag_Data = pd.concat([Data[['id','plylst_title', 'tags', 'songs']],
                            val_Data.iloc[val_TagIndex_list][['id','plylst_title', 'tags', 'songs']]], axis = 0)
plylst_tag_Data.head()

,id,plylst_title,tags,songs
0,61281,여행같은 음악,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,10532,요즘 너 말야,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,추억의 노래 ㅋ,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [115]:
len(plylst_tag_Data)

126676

In [116]:
tags = plylst_tag_Data['tags']
tags[:10]

0                                                  [락]
1                                             [추억, 회상]
2                                            [까페, 잔잔한]
3    [연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...
4                                                 [댄스]
5    [운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...
6                          [짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]
7                                        [잔잔한, 추억, 회상]
8                          [일렉트로니카, 포크, 메탈, 락, 댄스, 인디]
9                   [록, Metal, 이일우, M에센셜, 메탈, Rock, 락]
Name: tags, dtype: object

In [117]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [118]:
# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(tags, min_count=1)
model

2020-07-26 07:23:26,870 : INFO : collecting all words and their counts
2020-07-26 07:23:26,872 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-26 07:23:26,892 : INFO : PROGRESS: at sentence #10000, processed 41335 words, keeping 6233 word types
2020-07-26 07:23:26,909 : INFO : PROGRESS: at sentence #20000, processed 82859 words, keeping 9867 word types
2020-07-26 07:23:26,927 : INFO : PROGRESS: at sentence #30000, processed 123981 words, keeping 12964 word types
2020-07-26 07:23:26,945 : INFO : PROGRESS: at sentence #40000, processed 165354 words, keeping 15464 word types
2020-07-26 07:23:26,962 : INFO : PROGRESS: at sentence #50000, processed 206797 words, keeping 17704 word types
2020-07-26 07:23:26,979 : INFO : PROGRESS: at sentence #60000, processed 248011 words, keeping 19776 word types
2020-07-26 07:23:26,997 : INFO : PROGRESS: at sentence #70000, processed 289620 words, keeping 21732 word types
2020-07-26 07:23:27,016 : INFO : PROGRESS: at sent

In [119]:
# 모델 이름을 지정하고 저장한다.
model_name = 'KaKaoWords'
model.save(model_name)

2020-07-26 07:23:36,181 : INFO : saving Word2Vec object under KaKaoWords, separately None
2020-07-26 07:23:36,182 : INFO : not storing attribute vectors_norm
2020-07-26 07:23:36,183 : INFO : not storing attribute cum_table
2020-07-26 07:23:36,524 : INFO : saved KaKaoWords


In [120]:
import gensim 
import gensim.models as g

# 모델 불러오기
model_name = 'KaKaoWords'
model = g.Doc2Vec.load(model_name)

model.wv.vocab

2020-07-26 07:23:36,531 : INFO : loading Doc2Vec object from KaKaoWords
2020-07-26 07:23:36,747 : INFO : loading wv recursively from KaKaoWords.wv.* with mmap=None
2020-07-26 07:23:36,748 : INFO : setting ignored attribute vectors_norm to None
2020-07-26 07:23:36,748 : INFO : loading vocabulary recursively from KaKaoWords.vocabulary.* with mmap=None
2020-07-26 07:23:36,749 : INFO : loading trainables recursively from KaKaoWords.trainables.* with mmap=None
2020-07-26 07:23:36,750 : INFO : setting ignored attribute cum_table to None
2020-07-26 07:23:36,750 : INFO : loaded KaKaoWords


{'락': <gensim.models.keyedvectors.Vocab at 0x7f090b6b9990>,
 '추억': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa350>,
 '회상': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa390>,
 '까페': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa090>,
 '잔잔한': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa0d0>,
 '연말': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa150>,
 '눈오는날': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa490>,
 '캐럴': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa510>,
 '분위기': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa550>,
 '따듯한': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa250>,
 '크리스마스캐럴': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa290>,
 '겨울노래': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa2d0>,
 '크리스마스': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa650>,
 '겨울왕국': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa690>,
 '크리스마스송': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa6d0>,
 '댄스': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa3d0>,
 '운동

In [121]:
# 단어 사전 수
len(model.wv.vocab)

30197

In [122]:
# 단어 사전에서 상위 30개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]

['기분전환',
 '감성',
 '휴식',
 '발라드',
 '잔잔한',
 '드라이브',
 '힐링',
 '사랑',
 '새벽',
 '밤',
 '카페',
 '추억',
 '힙합',
 '팝',
 '이별',
 '설렘',
 '신나는',
 '회상',
 '매장음악',
 '여행',
 '인디',
 '가을',
 '겨울',
 '스트레스',
 '락',
 '슬픔',
 '랩',
 '뉴에이지',
 '봄',
 '여름']

In [123]:
# Counter로 자주 등장하는 단어 보기
from collections import Counter
dict(Counter(vocab).most_common(20))

{'기분전환': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa5d0>,
 '감성': <gensim.models.keyedvectors.Vocab at 0x7f090caa4b10>,
 '휴식': <gensim.models.keyedvectors.Vocab at 0x7f090b6aff10>,
 '발라드': <gensim.models.keyedvectors.Vocab at 0x7f090b639610>,
 '잔잔한': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa0d0>,
 '드라이브': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa450>,
 '힐링': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa590>,
 '사랑': <gensim.models.keyedvectors.Vocab at 0x7f090b6aac10>,
 '새벽': <gensim.models.keyedvectors.Vocab at 0x7f090ca9c5d0>,
 '밤': <gensim.models.keyedvectors.Vocab at 0x7f090ca9c150>,
 '카페': <gensim.models.keyedvectors.Vocab at 0x7f090b6af150>,
 '추억': <gensim.models.keyedvectors.Vocab at 0x7f090b6aa350>,
 '힙합': <gensim.models.keyedvectors.Vocab at 0x7f090ca9ca50>,
 '팝': <gensim.models.keyedvectors.Vocab at 0x7f090b6aaa10>,
 '이별': <gensim.models.keyedvectors.Vocab at 0x7f090b6aac50>,
 '설렘': <gensim.models.keyedvectors.Vocab at 0x7f090b6af450>,
 '신나는': <gensim.mode

In [124]:
# 가장 적게 등장하는 단어
min(vocab, key=vocab.get)

'이일우'

In [125]:
model.wv['여행']

array([ 0.36428314, -0.419554  , -0.2643931 , -0.5191465 ,  0.9339064 ,
       -0.5100837 , -0.7589976 ,  0.28691274, -1.6769532 ,  0.49631616,
        0.85663956, -0.11590077,  1.4249479 ,  1.2333671 ,  0.15906422,
       -0.44039375, -1.7513858 ,  0.6142299 ,  1.0420192 ,  0.91282904,
        1.5966096 ,  0.10985463,  0.87706596, -0.66061765,  0.49175447,
       -1.5947365 ,  0.49784496,  0.55498147,  0.04847426, -0.57071567,
       -0.4312156 ,  0.8296268 , -0.24980351, -0.7594368 ,  0.36963138,
       -0.13614897,  0.8132081 ,  0.30417162, -0.46723795,  0.83216614,
       -0.17602432, -0.7231104 ,  0.19944532, -0.5300707 , -0.68025976,
        1.1061321 ,  0.35149497, -1.020448  ,  0.67713857, -0.3172718 ,
        0.85550433, -0.35311377, -0.25855643,  0.146747  , -0.7929297 ,
       -0.34206048,  0.9847697 ,  0.19323666,  0.80670416,  0.6948436 ,
       -0.39505303,  0.73155975,  1.1928049 ,  0.49085677, -2.925717  ,
        0.17812793, -1.4940823 ,  0.02818119, -0.1719648 , -0.44

In [126]:
model.wv.most_similar('여행', topn = 20)

2020-07-26 07:23:54,509 : INFO : precomputing L2-norms of word weight vectors


[('자전거', 0.891935408115387),
 ('외출', 0.866045355796814),
 ('기분업', 0.8608759641647339),
 ('운전', 0.8509765863418579),
 ('공원', 0.850344181060791),
 ('청량한', 0.8478677272796631),
 ('즐거운', 0.8473473787307739),
 ('소소한일상', 0.8397858738899231),
 ('나들이', 0.8375917673110962),
 ('한강', 0.836681604385376),
 ('여름노래', 0.8307129144668579),
 ('상쾌', 0.8295570611953735),
 ('스트레스', 0.8290252685546875),
 ('통통튀는', 0.8284920454025269),
 ('여름', 0.8272495269775391),
 ('청량', 0.8233465552330017),
 ('청량감', 0.8226215839385986),
 ('라이딩', 0.8225150108337402),
 ('금요일', 0.8212008476257324),
 ('산뜻한', 0.8206052184104919)]

In [127]:
model.wv.rank('여행', '더위')

21

In [128]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('더위 상쾌 청량 발걸음'.split())

/home/data2/.local/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'발걸음'

In [129]:
model.wv.similarity('여행', '바다')

0.78893363

In [130]:
model.most_similar(positive=['여행','바다','노래'],topn=10)

/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('여름향기', 0.9449145197868347),
 ('차안에서', 0.9272115230560303),
 ('운전', 0.9238264560699463),
 ('출퇴근', 0.9194031357765198),
 ('여름노래', 0.910111665725708),
 ('기분', 0.9046749472618103),
 ('청량', 0.9043392539024353),
 ('무더위', 0.9038400650024414),
 ('자전거', 0.9032979011535645),
 ('통통튀는', 0.90160071849823)]

In [131]:
max(['발걸음','바다','청량'],key=vocab.get)

'청량'

# 태그 유사도 시각화

In [132]:
# 기본 글꼴 변경
import matplotlib as mpl
from sklearn.manifold import TSNE
from matplotlib import font_manager
import matplotlib.pyplot as plt
font_manager._rebuild()
plt.rc('font', family='NanumBarunGothic')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False


2020-07-26 07:23:59,976 : INFO : Could not open font file /usr/share/fonts/truetype/noto/NotoColorEmoji.ttf: In FT2Font: Could not set the fontsize
2020-07-26 07:24:00,506 : INFO : generated new fontManager


In [133]:
# Doc2Vec으로 만든 모델을 가져옴
vocab = list(model.wv.vocab)

# 모델의 단어를 피처로 지정해 준다.
X = model[vocab]
# print(X)

print('Feature의 길이 {}'.format(len(X)))
print(X[0][:10])
tsne = TSNE(n_components=2)

# 모든 단어를 그래프에 출력하면 글자가 너무 많이 겹치기 때문에 일부 단어에 대해서만 시각화
X_tsne = tsne.fit_transform(X[:5000,:])
X_tsne

/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Feature의 길이 30197
[ 0.5473364   0.26024628  0.23364253 -0.66881365 -0.96758807 -1.34867
  0.28107285 -0.19384477 -0.33537972 -0.2587759 ]


array([[ 65.81724 ,  24.56894 ],
       [ 53.246277, -13.196477],
       [ 53.12513 , -18.914326],
       ...,
       [ 49.52829 ,  45.08463 ],
       [-38.428387, -10.646131],
       [ 41.657368,  39.503708]], dtype=float32)

In [134]:
df = pd.DataFrame(X_tsne, index=vocab[:5000], columns=['x', 'y'])
df.shape

(5000, 2)

In [135]:
df.head(30)

,x,y
락,65.817238,24.568939
추억,53.246277,-13.196477
회상,53.125130,-18.914326
까페,41.177223,-17.932737
잔잔한,44.222416,-21.456032
연말,46.562798,-15.694201
눈오는날,45.393051,-14.266051
캐럴,45.830555,-13.097291
분위기,59.141769,-5.172223
따듯한,42.797379,-14.940247


In [136]:
%matplotlib qt5

fig = plt.figure(figsize = (24,16))
fig.set_size_inches(40, 20)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'])

for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=30)
plt.show()

# val.json

In [150]:
asdf = pd.read_json('val.json')
asdf.head(30)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
5,"[비, 분위기, 컨트리, 서정적인]",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
6,[],127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000
7,[],115317,,"[54457, 423722, 337682, 91214, 644980, 556444,...",3,2014-01-26 20:00:30.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000


In [138]:
len(asdf)

23015

In [140]:
# 빈 태그 갯수
cnt = 0
for i in range(len(asdf)):
    #print(asdf.iloc[i]['tags'])
    if asdf.iloc[i]['tags'] == []:
        cnt += 1
cnt

11410

In [141]:
max(asdf['id'].values), min(asdf['id'].values)

(153424, 3)

In [142]:
# 없을 때 처리
most_tags = [a for a, i in Counter(vocab).most_common(10)]
most_tags

['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']

In [143]:
asdf.iloc[5]['tags']

['비', '분위기', '컨트리', '서정적인']

In [151]:
cnt = 0
error_list = []
for i in range(len(asdf)):
    element = asdf.iloc[i]
   
    if element['tags'] == []:
        element['tags'] += most_tags
        print('{}번째 항목 : '.format(i), element['tags'])
    else:
        try:
#         print(i,element)
#         print(element['tags'])
            element_similarity = model.wv.most_similar(element['tags'], topn = 10)
            element_similarity_list = [a for a, i in element_similarity]
            element['tags'] +=(element_similarity_list)
            element['tags'] = element['tags'][:10]
            print('{}번째 항목 : '.format(i), element['tags'])
        except:
#             pass
            print('{}번째 항목 : 오류'.format(i))
            error_list.append(i)
            cnt +=1

/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance(

0번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
1번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
2번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
3번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
4번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
5번째 항목 :  ['비', '분위기', '컨트리', '서정적인', '센티멘탈', '흐림', '센치한날', '감성팝', '목소리깡패', '밤에']
6번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
7번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
8번째 항목 :  ['스트레스', '스트레스해소', 'BeaMiller', 'YOLO', '출퇴근', '운전', '흥폭발', '내적댄스', '기분업', '청량감']
9번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
10번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
11번째 항목 :  ['아침', '눈', '크리스마스', '눈오는날', '따듯한', '겨울노래', '연말', '추위', '겨울감성', '눈내리는']
12번째 항목 :  ['EDM', '일렉팝', '일렉트로니카', '일렉', '하우스', '딥하우스', 'electroni

190번째 항목 :  ['편하게', '별빛', '영감', '기운내', '뒹굴뒹굴하며듣기좋은', '버스안', '걸으며듣기좋은', '마감', '부드러운_잔잔한', '커피한잔에어울리는']
191번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
192번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
193번째 항목 :  ['일상', '나른한', '집', '아메리카노', '편안한', '커피', '여유', '오후', '포근한', '기분']
194번째 항목 :  ['운동', '다이어트', '라이딩', '헬스', '조깅', '휘트니스', '신남', '운동할때', '흥', '흥겨운']
195번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
196번째 항목 :  ['이별', '그리움', '쓸쓸함', '눈물', '쓸쓸하게', '공감', '쓸쓸', '추억에', '외로움', '쓸쓸할때']
197번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
198번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
199번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
200번째 항목 :  ['재연주', '드라이브엔', '응그래', 'CD샵대상', '달달하고신나는', 'TVN드라마', '빈지노의모든', '외적댄스', '여행휴식', '오오오옹']
201번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
202번째 항목 :  ['락', '추

380번째 항목 :  ['Relex', '팝송', 'popmusic', 'Acoustic', '국외', '팝뮤직', 'popsong', '취향', '해외팝', '팝송음악']
381번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
382번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
383번째 항목 :  ['랩', '힙합감성', 'Hip_Hop', '해외힙합', '국내힙합', '국힙', '외힙', '스웩', '비트좋은', '외국힙합']
384번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
385번째 항목 :  ['느낌적인', '분위기', 'mood', '분위기깡패', 'JAYJE', '사운드클라우드', '센티멘탈', '도시', '감성팝', '빈티지']
386번째 항목 :  ['주관적인', '추천선곡', '외국노래', '카카오톡', '하루를즐겁게', '블루스록', '데일리뮤직', '유튜브', '베스트', '최신음악']
387번째 항목 :  ['카페', '발라드', '기분전환', '사랑', '인디', '듣기좋은', '좋은노래', '고막남친', '멜로디', '음악']
388번째 항목 :  ['스트레스', '스트레스해소', 'BeaMiller', 'YOLO', '출퇴근', '운전', '흥폭발', '내적댄스', '기분업', '청량감']
389번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
390번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
391번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기

560번째 항목 :  ['봄', '팝들', '기분전환', '팝', '플레이리스트', '차안에서', '장르불문', '취저', '샤워', '매장노래']
561번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
562번째 항목 :  ['후추', '트랜디한노래', '너무많음', '그리움을담은노래', '중독되는', '리듬타기', 'uverworld', 'Top3', 'killuminati', '엑소제팬']
563번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
564번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
565번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
566번째 항목 :  ['설렘', '사랑', '연애', '계절', '설레임', '썸', '까페에서듣기좋은노래', '안락함', '달달', '사랑노래']
567번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
568번째 항목 :  ['기분전환', '팝송', '팝', '불금', '플레이리스트', 'SHOP', '스트레스', '차안에서', '운전', '출퇴근']
569번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
570번째 항목 :  ['음악한조각', '강추위', '지친하루', '포근한침대', '회사원', '당신의', '90년대댄스', '에이나우디', '울고_싶은_날', '띵곡EDM']
571번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
5

...

22157번째 항목 :  ['병아리', '노란', '나에게', '어디선가', '아이린', '다이아', '뭐먹지', '푸디토리움', 'Notorious', '시규어로스']
22158번째 항목 :  ['OST', '영화', '영화OST', '영화음악', '드라마', '드라마ost', '애니메이션', '사운드트랙', 'Animation', '감동']
22159번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22160번째 항목 :  ['슬픔', '새벽', '우울한', '쓸쓸', '감수성', '늦은밤', '별', '감성곡', '가을비', '센치한']
22161번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22162번째 항목 :  ['드라이브', '봄', '신나는', '즐거운', '외출', '차안에서', '운전', '한강', '기분업', '자전거']
22163번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22164번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22165번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22166번째 항목 :  ['Drum_and_Bass', '펑키', '트랜스', '일렉트로닉', '빅룸', 'Electronic', '누디스코', '전자음악', '커머셜', '트로피칼사운드']
22167번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22168번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴

22353번째 항목 :  ['데이트', '봄캐롤', '달달한', '벚꽃', '달콤', '달달', '달콤한', '소풍', '두근두근', '봄바람']
22354번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22355번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22356번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22357번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22358번째 항목 :  ['봄', '봄노래', '봄캐롤', '봄날', '봄바람', '달달한', '햇살', '꽃', '기분좋은', '피크닉']
22359번째 항목 :  ['가을', '가을밤', '가을감성', '낙엽', '가을비', '저녁', '감성자극', '흐린날', '가을노래', '듣기좋은']
22360번째 항목 :  ['Metal', '하드락', '해외록', '락음악', '록명곡', '락메탈', '락앤롤', '퀸', '척베리', '락밴드']
22361번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22362번째 항목 :  ['클럽', '일렉트로니카', 'EDM', '클럽음악', '일렉팝', '해외합합', '일렉', '들썩들썩', '흥', '하우스']
22363번째 항목 :  ['상쾌한', '시작', '맑은', '출근', '아침', '월요병', '월요일', '미세먼지', '밝은', '산뜻한']
22364번째 항목 :  ['사랑', '이별', '인디', '가사', '좋은노래', '평생소장', '공감', '인디음악', '감성충전', '노래']
22365번째 항목 :  ['포크', 

22565번째 항목 :  ['잔잔한', '잔잔', '감성음악', '감성자극', '센치', '카페에서', '봄비', '음악', '차분한', '비오는날']
22566번째 항목 :  ['풋풋한', '걸그룹', '추억', 'Pop', '솔로보컬', '히트곡', '하이틴', '인기가요', '인기곡', '역주행']
22567번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22568번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22569번째 항목 :  ['가을', '가을밤', '가을감성', '낙엽', '가을비', '저녁', '감성자극', '흐린날', '가을노래', '듣기좋은']
22570번째 항목 :  ['산책', '기분', '바람', '운전', '하늘', '일상', '한강', '나들이', '오후', '선선한']
22571번째 항목 :  ['쌀쌀', '추울때', '춥다', '늦가을', '추워', '쌀쌀한날', '녹는다', '차한잔', '추운겨울', 'newyear']
22572번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22573번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22574번째 항목 :  ['봄비', '카페', '날씨', '감성음악', '여유', '커피숍', '카페에서', '감성자극', '방콕', '따스한']
22575번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22576번째 항목 :  ['힙합', '감성힙합_2', '감성힙합', '힐링', '분위기', '스윗마인드', 'sweetmind', '멜로우', '센치한날', '분위기있는']
22

22747번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22748번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22749번째 항목 :  ['현장체험학습', 'Pop', '수학여행', '초등학생', '신나는', '팝음악', '빌보드핫100', '2010', '핫한', '볼륨업']
22750번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22751번째 항목 :  ['트랩', '쇼미더머니', 'SWAG', '스웩', '붐뱁', 'dope', '한국힙합', 'Hip_Hop', '래퍼', '해외힙합']
22752번째 항목 :  ['추천', '감성', '느낌있는', '잔잔', '몽환적', '듣기좋은', '귀르가즘', 'sweetmind', '분위기있는', '분위기깡패']
22753번째 항목 :  ['잔잔한', '잔잔', '감성음악', '감성자극', '센치', '카페에서', '봄비', '음악', '차분한', '비오는날']
22754번째 항목 :  ['새벽', '추억', '감수성', '가을비', '감성곡', '쓸쓸', '아련', '추억회상', '그립고', '감성충전']
22755번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22756번째 항목 :  ['스트레스', '록', '메탈', '락', 'Rock', '밴드', '얼터너티브', '스트레스해소', '파워풀', '모던락']
22757번째 항목 :  ['피아노', '연주음악', '피아노연주곡', '뉴에이지', '낮잠', '독서', '책', '명상', '태교', '연주곡']
22758번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '

22948번째 항목 :  ['아침', '드라이브', '기분전환', '주말', '금요일', '월요병', '외출', '불금', '한강', '출근길']
22949번째 항목 :  ['위클리초이스', '웹진이즘', 'izm', '웹진웨이브', 'weiv', '아코디언', '로스엔젤레스', '출근길_기분좋은', '골든브라더스', '펜타포트']
22950번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22951번째 항목 :  ['힙합', '국내힙합', '외국힙합', '외힙', '비트', 'HipHop', 'Rap', '국힙', '힙합추천', '랩힙합']
22952번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22953번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22954번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22955번째 항목 :  ['걸그룹', '상큼', '청량한케이팝', '여자아이돌', '보이그룹', '댄스곡', '아이돌', '남자아이돌', '미스터트롯', '트로트']
22956번째 항목 :  ['가요', '아련', '노랫말을듣는다', '가창력', '리메이크', '명곡들', 'ralalra', '7080', '추억회상', '옛날노래']
22957번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22958번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']
22959번째 항목 :  ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴'

In [153]:
cnt, error_list

(0, [])

In [162]:
len(asdf['songs'])

23015

In [209]:
for i in range(len(asdf)):
    element = asdf.iloc[i]
    element['songs'] += [i for i in range(300)]
    element['songs'] = list(set(element['songs']))[:100]

/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/data2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [211]:
asdf.head(30)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",131447,앨리스테이블,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1,2014-07-16 15:24:24.000
2,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
5,"[비, 분위기, 컨트리, 서정적인, 센티멘탈, 흐림, 센치한날, 감성팝, 목소리깡패...",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
6,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000
7,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",115317,,"[54457, 423722, 337682, 91214, 644980, 556444,...",3,2014-01-26 20:00:30.000
8,"[스트레스, 스트레스해소, BeaMiller, YOLO, 출퇴근, 운전, 흥폭발, ...",80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",127,2017-02-09 17:33:45.000
9,"[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]",142007,기분 좋은 재즈와 함께 만드는 달달한 하루,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0,2015-06-22 09:11:02.000


In [212]:
len(asdf)

23015

In [213]:
result = asdf[['id', 'songs', 'tags']]
result.head(20)

,id,songs,tags
0,118598,"[373313, 151080, 275346, 696876, 165237, 52593...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
1,131447,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
2,51464,"[529437, 516103, 360067, 705713, 226062, 37089...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
3,45144,"[589668, 21711, 570151, 320043, 13930, 599327,...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
4,79929,"[672718, 121924, 102694, 683657, 201558, 38511...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
5,138538,"[173033, 138237, 63140, 15733, 461780, 308519,...","[비, 분위기, 컨트리, 서정적인, 센티멘탈, 흐림, 센치한날, 감성팝, 목소리깡패..."
6,127575,"[227044, 424672, 515592, 22408, 57893, 153714,...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
7,115317,"[54457, 423722, 337682, 91214, 644980, 556444,...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"
8,80810,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[스트레스, 스트레스해소, BeaMiller, YOLO, 출퇴근, 운전, 흥폭발, ..."
9,142007,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[락, 추억, 회상, 까페, 잔잔한, 연말, 눈오는날, 캐럴, 분위기, 따듯한]"


In [223]:
# json 파일 형태로 수정
import json
result_list = list()
for i in range(len(result)):
    result_dict = dict()
    result_dict['id'] = int(result.iloc[i]['id'])
    result_dict['songs'] = list(set(result.iloc[i]['songs']))[:100]
#     print(i,'번째 저장' , len(result_dict['songs']))
    result_dict['tags'] = list(result.iloc[i]['tags'])[:10]
    result_list.append(result_dict)
#     print(len(set(result_dict['songs'])))
    
result_list[:10]

[{'id': 118598,
  'songs': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   104455,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   37,
   38,
   39,
   36,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99],
  'tags': ['락', '추억', '회상', '까페', '잔잔한', '연말', '눈오는날', '캐럴', '분위기', '따듯한']},
 {'id': 131447,
  'songs': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,

In [224]:
# 파일 저장
with open('results.json', 'w', encoding = 'utf-8') as f:
    json.dump(result_list, f, ensure_ascii = False, indent ='\t')


In [205]:
# 파일 읽기
with open('results.json') as json_file:
    json_data = json.load(json_file)
print(json_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

